# Стохастические градиентные методы

## На прошлой лекции

- Быстрый градиентный метод
- Сходимость и отличие от метода тяжёлого шарика
- Метод Ньютона
- Вычислительная сложность и скорость сходимости

## План на сегодня

- Стохастический градиент: что это и чем отличается от обычного?
- Когда и как именно можно его использовать для решения задачи?
- Особенности адаптированного подбора шага в таких методах
- Сравнение сходимости с детерминированным методами

## Постановка задачи

- Что мы рассматривали ранее

$$ \min_x f(x) $$

- Что будем рассматривать сегодня

$$ \min_x \frac{1}{N} \sum_{i=1}^N f_i(x) $$

- Теперь знаем как функция выглядит
- Больше информации о функции - больше возможностей к построению более эффективных методов

## Стохастический градиент

- Точный градиент

$$ \frac{1}{N} \sum_{i=1}^N f_i'(x) $$

- **Стохастический** градиент 

$$ \frac{1}{|\mathcal{I}|} \sum_{i \in \mathcal{I}} f_i'(x) $$

- Идея: идея используем только некоторые слагаемые
- Как вы думаете как выбирать какие $f_i$ использовать и почему?

## Стохастический градиентный спуск (SGD)

$$ x_{k+1} = x_k - \alpha_k g_k, \qquad g_k = \frac{1}{|\mathcal{I}_k|} \sum_{i \in \mathcal{I}_k} f_i'(x) $$

- Сходимость определяется через усреднение по истории
- Для выпуклых функций сходимость $\mathcal{O}(1 / \sqrt{k})$
- Для сильно выпуклых функций $\mathcal{O}(1 / k)$

- Все скорости сходимости стали **медленнее**, чем были для классического градиентного спуска
- Причина: направление движения в SGD **не обязано** быть направлением убывания
- Используется постоянный шаг или проводится изменение по некоторому расписанию: много эвристик

## Какие плюсы есть у неточного вычисления градиента?

- Вычисляется быстрее
- Если у функции много локальных минимумов, то неточный градиент может приводить к посещению большого числа локальных минимумов

## Применение к обучению нейросетей

- Напомним, что нейросеть - это суперпозиция простых функций (линейных и нелинейных), у которых есть параметры
- Для заданной функции потерь, которая как раз таки и представляется в виде суммы конечного, но большого числа функций, ищутся оптимальные параметры
- Минимизация функции потерь - формализация процесса обучения нейросети
- Каждое слагаемое в функции потерь - это величина ошибки при обработке одного объекта, например одного изображения или анкеты одного потенциального заёмщика
- Стандартные архитектуры не являются выпуклыми по параметрам, поэтому возможность "побывать" в различных локальных минимумах - важное свойство методов из этого класса  

## Недостатки SGD

- Сходимость можно измерять только на основании стабилизации значения целевой функции
- Небольшой размер множества индексов приводит к высокой вариативности направлений на различных итерациях. Формально, это свойство называется большой дисперсией в оценке градиента
- Неудачный размер шага может приводить либо к расходимости либо к очень медленной сходимости метода, так как это константа для всех элементов вектора параметров  

## Ускоренные стохастические методы

- По аналогии с ускорением классического градиентного спуска можно ускорять и его стохастическую модификацию
- Однако улучшения теоретических оценок не происходит хотя на практике методы работают быстрее
- Каким образом можно улучшить теоретические оценки?

## Метод усреднения стохастических градиентов (Stochastic averaged gradient method)

- Сначала вычислим все $g_i = f_i(x_0)$ и **сохраним**
- Далее на итерации $k+1$ вычислим $f_{i_k}(x_k) = h_k$


$$ x_{k+1} = x_k - \alpha \left(\frac1N h_k - \frac1N g_{i_k} + \frac{1}{N}\sum_{i=1}^N g_i\right) $$
$$ g_{i_k} = h_k $$

- Для такого метода показано [тут](https://arxiv.org/pdf/1309.2388.pdf), что оценки сходимости совпадают со сходимостью классического градиентного спуска 

## Адаптивные методы

- Размер шага свой для каждого элемента
- Размер шага подбирается на основе предыдущих значений стохастического градиента
- Скользящее среднее от вычисленных стохастических градиентов повышает устойчивость всего процесса

### Adagrad

\begin{align*}
& g_k = \sum_{i \in \mathcal{I}} f'_i(x_k)\\
& r = r + g_k \cdot g_k\\
& h_k = -\frac{\varepsilon}{\delta + \sqrt{r}} \cdot g_k\\
& x_{k+1} = x_k + h_k
\end{align*}


- Все операции выполняются поэлементно
- Задаётся базовый размер шага $\varepsilon$
- Эквивалентно диагональному шкалированию вектора градиента
- Какой недостаток у такого метода?

### Adam

\begin{align*}
& g_k = \sum_{i \in \mathcal{I}} f'_i(x_k)\\
& r = \rho_2r + (1 - \rho_2) g_k \cdot g_k\\
& s = \rho_1 s + (1 - \rho_1) g_k\\
& \hat{s} = \frac{s}{1 - \rho_1^k}\\
& \hat{r} = \frac{r}{1 - \rho_2^k}\\
& h_k = -\frac{\varepsilon \hat{s}}{\delta + \sqrt{\hat{r}}} \\
& x_{k+1} = x_k + h_k
\end{align*}

- Типичные значения параметров $\rho_1 = 0.9, \rho_2 = 0.999$
- Для обьновления параметров используется сглаженные версии градиентов и их поэлементных квадратов
- Устойчиво работает на большом классе задач обучения нейросетей

### Сходимость адаптивных методов

- Законченная теория сходимости таких методов на данный момент отсутствует
- Множество эмпирических фактов говорит, что такие методы требуют существенно меньше времени
- Однако есть и недостатки, связанные с возможностью переобучения 

## Другие модификации

<img src="optax_common.png" width=400>

- Список взят из документации к [optax](https://optax.readthedocs.io/en/latest/)
- Как вы думаете, что можно улучшать в рассмотренных выше методах?

## Выводы

- Постановка задачи пригодная для использования стохастического градиента
- Стохастические градиентные методы - мощный инструмент для обучения нейросетей
- Достоинства и недостатки таких методов
- Адаптивные методы выбора шага